The goal of this notebook is to save the new data in order to be able to get constraints 

In [1]:
import datetime
import numpy as np
import os
import pandas as pd
from collections import defaultdict
import matplotlib.pyplot as plt
#Created a 01/27/2022 
print(f'Last edited: {datetime.datetime.now()}')


Last edited: 2022-08-30 12:38:40.684658


In [2]:
filepath  = 'Data/IGF/EXP129_RAW.xlsx'
xls = pd.ExcelFile(filepath)
Data_Dict = defaultdict(lambda: 'Not present')
Sheet_labels = ['Sheet2' ,'0pM','10pM','15pm','20pM','25pM', '50pM', '250pM']
# conc
nSheets = len(Sheet_labels)
for i in range(nSheets): 
    sl = Sheet_labels[i]
    df = pd.read_excel(xls, sl ,header=None)
    data = df.to_numpy()
    Data_Dict[sl] = data
    print(f'saved {sl} to dictionary')



saved Sheet2 to dictionary
saved 0pM to dictionary
saved 10pM to dictionary
saved 15pm to dictionary
saved 20pM to dictionary
saved 25pM to dictionary
saved 50pM to dictionary
saved 250pM to dictionary


In [6]:
### Getting the scaling factor
df = pd.read_excel(xls, 'Sheet2' ,header=None)
bg_array = df.to_numpy()[:,1]
# bg_matrix = np.transpose(np.tile(bg_array, (300,1)))

# conc_labels = ['10pM','15pm','20pM','50pM']
conc_labels = ['10pM','15pm','20pM','25pM', '50pM', '250pM']
# conc_labels = Sheet_labels
# conc
bgg = 0 
ncc = 0
# Means_dict = defaultdict(lambda: 'Not present')
# Var_dict = defaultdict(lambda:'Not present')
# SecondMoment_dict = defaultdict(lambda:'Not present')
for key in conc_labels: 
    print(key)
    matrix = Data_Dict[key][:,1:]
    ncells = matrix.shape[1]
    bg_matrix = np.transpose(np.tile(bg_array, (ncells,1)))
#     print(matrix.shape)
    ## subtract 
    data = matrix - bg_matrix
#     if key 
    bgg += np.sum(data[0,:])
    ncc += len(data[0,:]) 
#     Means_dict[key] = np.mean(data, axis = 1)
#     SecondMoment_dict[key] = np.mean(data**2, axis = 1)
#     Var_dict[key] = np.var(data, axis = 1)
    
flour_noligand = bgg/ncc
print(flour_noligand)
n_foxO = 710*2/3 
au_foxO = flour_noligand
# scale factor 
sc = n_foxO/au_foxO
print(f'sc = {sc}')

10pM
15pm
20pM
25pM
50pM
250pM
22180.27389783454
sc = 0.021340283511086165


In [7]:
### Removing the background and getting the shift
df = pd.read_excel(xls, 'Sheet2' ,header=None)
bg_array = df.to_numpy()[:,1]
# bg_matrix = np.transpose(np.tile(bg_array, (300,1)))

# conc_labels = ['10pM','15pm','20pM','50pM']
# conc
bgg = 0 
ncc = 0

Means_dict = defaultdict(lambda: 'Not present')
for key in conc_labels: 
    print(key)
    matrix = Data_Dict[key][:,1:]
    ncells = matrix.shape[1]
    bg_matrix = np.transpose(np.tile(bg_array, (ncells,1)))
#     print(matrix.shape)
    ## subtract 
    data = (matrix - bg_matrix)*sc
#     if key 
#     bgg += np.sum(data[0,:])
#     ncc += len(data[0,:]) 
    Means_dict[key] = np.mean(data, axis = 1)

10pM
15pm
20pM
25pM
50pM
250pM


In [8]:
means_matrix = np.zeros((len(conc_labels), 61))
i=0
for key in conc_labels:
    means_matrix[i,:] = Means_dict[key]
    i+=1 
np.shape(means_matrix)

(6, 61)

In [9]:
data_shift_dict = defaultdict(lambda: 'not present')
data_shift = (np.average(means_matrix[:,0]) - means_matrix[:,0])
for i in range(np.shape(means_matrix)[0]):
    
    means_matrix[i] = means_matrix[i] + data_shift[i]

### Saving the data with bg removal, shift added and scaling .

In [11]:
df = pd.read_excel(xls, 'Sheet2' ,header=None)
bg_array = df.to_numpy()[:,1]

# conc_labels = ['10pM','15pm','20pM','50pM']
conc_labels = ['10pM','15pm','20pM','25pM', '50pM', '250pM']

# conc
bgg = 0 
ncc = 0
ProDataDict= defaultdict(lambda: 'not present')
i=0
for key in conc_labels: 
    print(key)
    # load original data 
    
    matrix = Data_Dict[key][:,1:]
    ncells = matrix.shape[1]
    bg_matrix = np.transpose(np.tile(bg_array, (ncells,1)))
    # process the data 
    data =( ((matrix - bg_matrix)*sc ) + data_shift[i])
    print(data.shape)
    if key == '15pm':
        key='15pM'
    ProDataDict[key]= data
    i+=1
    
# bgg/ncc




10pM
(61, 301)
15pm
(61, 300)
20pM
(61, 300)
25pM
(61, 300)
50pM
(61, 300)
250pM
(61, 300)


In [12]:
ProDataDict

defaultdict(<function __main__.<lambda>()>,
            {'10pM': array([[416.00168574, 594.29975447, 487.29957295, ..., 400.82874416,
                     416.59921368, 416.89797764],
                    [429.40710309, 541.10214699, 466.47517555, ..., 378.95867287,
                     378.93733258, 419.14242672],
                    [408.97234878, 582.83163854, 467.16730191, ..., 371.4134498 ,
                     388.40031547, 414.02999597],
                    ...,
                    [402.06352345, 464.20642904, 519.96858985, ..., 401.14589126,
                     345.31970959, 430.53146165],
                    [393.57006013, 467.32207994, 527.54436001, ..., 394.08222693,
                     349.05422872, 424.10800583],
                    [407.26549193, 484.09051257, 524.10354415, ..., 389.76645945,
                     347.85414264, 420.15502317]]),
             '15pM': array([[486.98386909, 573.00655193, 487.77345958, ..., 500.64165054,
                     456.14715942, 416.

In [14]:
## Save the data 
# ProDataDict = dict(ProDataDict)
np.save('ProDataDict_allconc.npy', dict(ProDataDict), allow_pickle=True)

In [15]:
6*7

42